In [ ]:
from pygoridge import RPC, SocketRelay
import numpy as np
from pygoridge.constants import PayloadType
import pandas as pd
import shap
import json

# Run go run .\cmd\generate_dataset\main.go at least once before running this notebook, so that "dataset_AI_Random_7.csv" exists!
real_game_dataset = pd.read_csv('dataset_AI_Random_7.csv')

# Run go run .\cmd\inference-rpc-bridge\main.go before running this notebook! 
rpc = RPC(SocketRelay("127.0.0.1", 6001))

# Little "hack" so that numpy writes to memory instead of saving to disk.
# I did not find any way to get this format as a variable instead of a file on disk
# And I did not want to have the overhead of writing to disk
class memfile:
    def __init__(self):
        self.b = []
    def write(self, b):
        self.b += b
    def asbytes(self):
        return bytes(self.b)

def go_ur_ai_numpy(x):
    mm = memfile()
    np.save(mm, x)
    return np.array(json.loads(rpc("GoUr.InferNumpy", mm.asbytes(), PayloadType.PAYLOAD_RAW)))


In [ ]:
real_game_dataset.columns = [f'{col}_pos_{i+2}' for i, col in enumerate(real_game_dataset.columns)]

n_samples = 100
bg_values = real_game_dataset.sample(10)
data_sample = real_game_dataset.sample(n_samples)

explainer = shap.KernelExplainer(go_ur_ai_numpy, bg_values, link="logit")
shap_values = explainer.shap_values(data_sample, n_samples=n_samples)

## Force plot

This shows the shap force plot of a single dice throw

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[18], data_sample.iloc[18], link="logit")

In [ ]:
df_shap = pd.DataFrame(shap_values)
avg_abs_shap_values = pd.Series(df_shap.abs().mean())
var_shap_values = pd.Series(df_shap.var())
min_shap_values = pd.Series(df_shap.min())
max_shap_values = pd.Series(df_shap.max())

shap_columns = pd.DataFrame({
    'Column': data_sample.columns,
    'Average Absolute Shap': avg_abs_shap_values,
    'Variance Shap': var_shap_values,
    'Min Shap': min_shap_values,
    'Max Shap': max_shap_values,
}).set_index('Column').sort_values('Average Absolute Shap', ascending=False)
shap_columns

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values, data_sample, link='logit', feature_display_range=slice(None, -100, -1))

## Analysis of strongly positive moves

In [ ]:
y = go_ur_ai_numpy(data_sample.values)

In [ ]:
y = pd.Series(y)

In [ ]:
good_scores_mask = y > 0.1
good_scores_mask.sum()

In [ ]:
good_scores = data_sample.reset_index(drop=True)[good_scores_mask]
good_scores

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values[good_scores_mask], good_scores, feature_display_range=slice(None, -100, -1), link="logit")